# NVIDIA Driver

You'll need a functional nvidia compute capability 5.3 or better card. This basically means
Testa P series or Geforce GTX 10 series or better. See https://developer.nvidia.com/cuda-gpus
for more details.

Ensure nvidia-smi can see your gpu. Should look something like this:
    
```
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   35C    P8     7W / 120W |    454MiB /  6075MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
```

In [1]:
!nvidia-smi

Fri Jan  3 21:18:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   34C    P8     7W / 120W |    651MiB /  6075MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# TensorFlow GPU support

Ensure tensorflow has functional gpu support. You should have a GPU line 
referencing the GPU identified by nvidia-smi above.

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
    
with tf.device(tf.test.gpu_device_name()):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
    # Creates a session with log_device_placement set to True.
    sess = tf.Session(config=tf.ConfigProto())
    # Runs the op.
    r = sess.run(c)
    print(r)
    import numpy as np
    assert np.all(r == np.array([[22., 28.], [49., 64.]]))

Default GPU Device: /device:GPU:0
[[22. 28.]
 [49. 64.]]


# OpenCV CUDA Support

Ensure opencv was built with gpu support

In [3]:
import cv2
print(cv2.getBuildInformation())


General configuration for OpenCV 4.2.0 =====================================
  Version control:               unknown

  Extra modules:
    Location (extra):            /opencv_contrib-4.2.0/modules
    Version control (extra):     unknown

  Platform:
    Timestamp:                   2020-01-03T14:38:08Z
    Host:                        Linux 5.3.0-24-generic x86_64
    CMake:                       3.10.2
    CMake generator:             Unix Makefiles
    CMake build tool:            /usr/bin/make
    Configuration:               RELEASE

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2 AVX512_SKX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (2 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (1 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (5 files):   

In [4]:
import numpy as np
import cv2 as cv
import os

if not cv.cuda.getCudaEnabledDeviceCount():
    print("No CUDA-enabled OpenCV devices found")
else:
    npMat = (np.random.random((128, 128, 3)) * 255).astype(np.uint8)
    cuMat = cv.cuda_GpuMat()
    cuMat.upload(npMat)
    print ("CUDA upload succeeded: " + str(np.allclose(cuMat.download(), npMat)))
    
    npMat1 = np.random.random((128, 128, 3)) - 0.5
    npMat2 = np.random.random((128, 128, 3)) - 0.5

    cuMat1 = cv.cuda_GpuMat()
    cuMat2 = cv.cuda_GpuMat()
    cuMat1.upload(npMat1)
    cuMat2.upload(npMat2)
    cuMatDst = cv.cuda_GpuMat(cuMat1.size(),cuMat1.type())

    print ("CUDA arithmetic succeeded: " + str(np.allclose(cv.cuda.add(cuMat1, cuMat2).download(),
                                     cv.add(npMat1, npMat2))))

CUDA upload succeeded: True
CUDA arithmetic succeeded: True


# Working Directories 

In [5]:
import os
from os import makedirs
from os.path import isdir

readable = ["../data"]
writable = [
    "../src/tf_configs",
    "../src",
    "../scratch",
    "../scratch/models",
    "../scratch/tf_records/",
    "../scratch/train/checkpoints"
]

for f in writable:
    makedirs(f, exist_ok=True)
    
for f in readable + writable:
    print("\tdirectory:", isdir(f), "readable:", os.access(f, os.R_OK), "writable:", os.access(f, os.W_OK), "\t", f)

	directory: True readable: True writable: False 	 ../data
	directory: True readable: True writable: True 	 ../src/tf_configs
	directory: True readable: True writable: True 	 ../src
	directory: True readable: True writable: True 	 ../scratch
	directory: True readable: True writable: True 	 ../scratch/models
	directory: True readable: True writable: True 	 ../scratch/tf_records/
	directory: True readable: True writable: True 	 ../scratch/train/checkpoints


# Download Models

In [6]:
! cd ../scratch/models && curl -sL http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz | tar -xzv

faster_rcnn_resnet101_coco_2018_01_28/
faster_rcnn_resnet101_coco_2018_01_28/model.ckpt.index
faster_rcnn_resnet101_coco_2018_01_28/checkpoint
faster_rcnn_resnet101_coco_2018_01_28/pipeline.config
faster_rcnn_resnet101_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_resnet101_coco_2018_01_28/model.ckpt.meta
faster_rcnn_resnet101_coco_2018_01_28/saved_model/
faster_rcnn_resnet101_coco_2018_01_28/saved_model/saved_model.pb
faster_rcnn_resnet101_coco_2018_01_28/saved_model/variables/
faster_rcnn_resnet101_coco_2018_01_28/frozen_inference_graph.pb
